# DAT300 CA1: Selecting features for nuclear forensics 


In [1]:
%matplotlib inline

import time

import pandas
import numpy
import sklearn
import sklearn.model_selection
import sklearn.ensemble
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.feature_selection
import sklearn.linear_model

# Dataset

In [2]:
# Return a combined DataFrame, and a dictionary of {featuretype: columns}
def load_data(filename):
    def load_sheet(name):
        df = file.parse(name)
        if 'Index' in df.columns:
            # we index based on row number
            del df['Index']
        else:
            # remove duplicated labels
            del df['Labels']
        return df
    
    file = pandas.ExcelFile(filename)
    dfs = { sheet: load_sheet(sheet) for sheet in file.sheet_names}
    
    # AMT is missing names for columns
    combine = [dfs[k] for k in dfs.keys() if k not in ('AMT')]
    
    combined = dfs['AMT'].copy().add_suffix('-AMT')
    combined = combined.join(combine, lsuffix='', rsuffix='')
    
    feature_categories = {}
    for category, df in dfs.items():
        columns = df.columns
        feature_categories[category] = columns
    
    return combined, feature_categories

fulldata, categories = load_data('DAT300 - CA 1_NEW.xlsx')
assert fulldata.shape[0] == 120, fulldata.shape
assert numpy.count_nonzero(fulldata.Labels.isnull()) == 0, numpy.count_nonzero(fulldata.Labels.isnull())


In [3]:
categories.keys()

dict_keys(['Target', 'AMT', 'WT_originals', 'WT-LLL', 'WT-LLH', 'WT_LHL', 'WT_LHH', 'LBP'])

In [4]:
fulldata.head()

,1-AMT,2-AMT,3-AMT,4-AMT,5-AMT,6-AMT,7-AMT,8-AMT,9-AMT,10-AMT,...,"lbp_24_(24,3)","lbp_25_(24,3)","lbp_2_(24,3)","lbp_3_(24,3)","lbp_4_(24,3)","lbp_5_(24,3)","lbp_6_(24,3)","lbp_7_(24,3)","lbp_8_(24,3)","lbp_9_(24,3)"
0,1.55144,1.51980,1.45847,1.43940,1.43263,1.44893,1.50741,1.54977,1.59866,1.63075,...,53695,468638,17261,15439,18953,22857,24262,26894,27454,29276
1,1.51458,1.47539,1.45449,1.46195,1.44090,1.47846,1.51607,1.53459,1.51287,1.54192,...,53047,462418,16941,15294,18823,22369,24826,27250,28437,30374
2,1.50565,1.46602,1.44400,1.45143,1.42508,1.41574,1.44243,1.47648,1.44198,1.45998,...,53917,469225,17328,15441,19056,22997,24702,26641,27585,29121
3,1.53664,1.50323,1.46508,1.42571,1.43470,1.42078,1.44322,1.45240,1.46457,1.51837,...,53643,465916,16694,15318,18830,22634,24608,26706,27805,29507
4,1.52808,1.47522,1.45618,1.37537,1.37561,1.39346,1.42378,1.46455,1.47553,1.46507,...,48987,456795,15882,14244,17120,20866,23471,26877,28607,31338


In [5]:
fulldata.shape

(120, 1426)

In [6]:
def get_XY(data, feature_columns=None):
    all_feature_columns = list(set(data.columns) - set(['Labels']))
    
    use = data[data.Labels.notna()]
    X = use[all_feature_columns].astype(float)
    if feature_columns is not None:
        feature_columns = list(feature_columns)
        X = X.iloc[:,feature_columns]
    y = use.Labels
    return X,y

# Models

In [7]:
kNN = ( sklearn.neighbors.KNeighborsClassifier(), {
       'n_neighbors': [1,2,3,4],
})
RandomForest = ( sklearn.ensemble.RandomForestClassifier(), {
    'n_estimators': [10, 30, 100],
    'min_samples_leaf': [0.01, 0.1],
})
Logistic = ( sklearn.linear_model.LogisticRegression(multi_class='ovr', solver='liblinear'),  {
    'C': [ 0.75, 0.5, 0.25 ],
})

# TODO: add GBT
# MAYBE: add SVM polynomial

models = [
    kNN,
    RandomForest,
    Logistic,
]


## Model evaluation


In [8]:
def evaluate_candidate(dataset, cand, gridcv=5, n_tests=100, seed=0, evalcv=3):
    model, params, features = cand

    # Reduce to subset of features
    X, y = get_XY(dataset, features)
    X = sklearn.preprocessing.RobustScaler().fit_transform(X)
    
    # Decide hyperparameters
    gridsearch_start = time.time()
    grid = sklearn.model_selection.GridSearchCV(model, params, cv=gridcv,
                                                iid=False, refit=True, return_train_score=True)
    grid.fit(X, y)
    estimator = grid.best_estimator_
    
    gridsearch_time = time.time() - gridsearch_start
    
    # Evaluate on a range of test-train splits
    numpy.random.seed(seed)
    test_scores = numpy.array([])
    train_scores = numpy.array([])

    evaluate_start = time.time()
    for rng in numpy.random.randint(0, 1000, size=n_tests):
        X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.4, random_state=rng)

        estimator.fit(X_train, Y_train)

        # FIXME: calculate score across whole test/trainset
        test = sklearn.model_selection.cross_val_score(estimator, X_test, Y_test, cv=evalcv)
        train = sklearn.model_selection.cross_val_score(estimator, X_train, Y_train, cv=evalcv)
        
        test_scores = numpy.concatenate([test_scores, test]) 
        train_scores = numpy.concatenate([train_scores, train])
    evaluate_time =  time.time() - evaluate_start
    
    return test_scores, train_scores, grid.cv_results_, gridsearch_time, evaluate_time

def test_evaluate_candidate():
    all_features = None
    candidate = (sklearn.ensemble.RandomForestClassifier(n_estimators=10), {}, all_features)
    r = evaluate_candidate(fulldata, candidate, n_tests=10)
    assert len(r) == 5
                       
test_evaluate_candidate()

# Feature selection

### Random selection


In [9]:
def random_candidates(models, feature_lengths, n_random=5, max_features=1425):
    candidates = []
    for n_features in feature_lengths:
        random_features = numpy.random.choice(range(max_features), size=(n_random, n_features), replace=False)
        for features in random_features:
            for (m,p) in models:
                candidates.append(( m, p, features, 'random' ))

    return candidates
random_candidates(models, (1,), 5);

### Univariate feature scoring

In [10]:
def feature_score_rf(X, y, n_reps=30):
    importances = numpy.ndarray((n_reps, X.shape[1]))
    # RF tends to ignore redundant features, so average over many runs
    for i in range(0, n_reps):
        rf = sklearn.ensemble.RandomForestClassifier(n_estimators=50, random_state=i)
        rf.fit(X, y)
        s = rf.feature_importances_
        importances[i] = s
        assert numpy.isclose(numpy.sum(s), 1.0), numpy.sum(s)
    
    return numpy.mean(importances, axis=0)

def extract_feature_scores(X, y, scorers):
    ret = []
    X = sklearn.preprocessing.MinMaxScaler().fit_transform(X)
    for f in scorers:
        scores = numpy.array(f(X, y))
        if scores.shape[0] == 2:
            scores = scores[0] # drop p-values
        # normalize it
        scores = scores/scores.sum()
        ret.append(scores)
    return numpy.array(ret)

def select_k_best(scores, n):
    return numpy.argsort(scores)[:n]

def univariate_candidates(models, features_length, scores):
    candidates = []
    for n_features in features_length:
        
        for (m,p) in models:
            # individual scoring functions
            for scores in feature_scores:
                cand = ( m, p, select_k_best(scores, n_features), 'k-best' )
                candidates.append(cand)

            # combined univariate score
            scores = numpy.mean(feature_scores, axis=0)
            assert scores.shape[0] == 1425
            candidates.append((m, p, select_k_best(scores, n_features), 'k-best-mean'))

    return candidates

feature_scorers = [
    sklearn.feature_selection.mutual_info_classif,
    sklearn.feature_selection.f_classif,
    sklearn.feature_selection.chi2,
]
feature_scores = extract_feature_scores(*get_XY(fulldata), feature_scorers)
assert feature_scores.shape[1] == 1425
univariate_candidates(models, (1,), feature_scores);

In [11]:
# TODO: plot feature scores, each type

In [12]:
feature_scores.shape

(3, 1425)

## Sequential Forward Selection


In [29]:
from mlxtend.feature_selection import SequentialFeatureSelector

def sfs_candidates(dataset, models, features_length):
    candidates = []
    
    estimators = {
        'RF': sklearn.ensemble.RandomForestClassifier(n_estimators=1, max_depth=5),
        'kNN1': sklearn.neighbors.KNeighborsClassifier(n_neighbors=1),
    }
    for name, estimator in estimators.items():
        max_features = max(features_length)
        sfs = SequentialFeatureSelector(estimator, k_features=max_features, forward=True, floating=False,
                                        cv=5, scoring='accuracy', verbose=1, n_jobs=2)
        X, y = get_XY(dataset)
        sfs.fit(X, y)

        # Also evaluate shorter combinations of the best K
        for n_features in features_length:
            features = sfs.k_feature_idx_[:n_features]
    
            for (m, p) in models:
                candidates.append((m, p, features, 'sfs'))
        
    return candidates


In [30]:
def all_features_candidates(models):
    candidates = []
    for (m, p) in models:
        candidates.append(( m, p, None, 'all'))
    return candidates

In [31]:
feature_lengths = (1, 2, 3, 5)

candidates = [] + \
    all_features_candidates(models) + \
    sfs_candidates(fulldata, models, feature_lengths) + \
    univariate_candidates(models, feature_lengths, feature_scores) + \
    random_candidates(models, feature_lengths, n_random=3)

#numpy.random.shuffle(candidates)

#for cand in candidates:
#    print('cand', cand[2], type(cand[0]), cand[3])

len(candidates)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  80 tasks      | elapsed:   14.5s
[Parallel(n_jobs=2)]: Done 680 tasks      | elapsed:   35.5s
[Parallel(n_jobs=2)]: Done 1425 out of 1425 | elapsed:  1.0min finished
Features: 1/5[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  88 tasks      | elapsed:    3.2s
[Parallel(n_jobs=2)]: Done 388 tasks      | elapsed:   14.0s
[Parallel(n_jobs=2)]: Done 888 tasks      | elapsed:   33.2s
[Parallel(n_jobs=2)]: Done 1424 out of 1424 | elapsed:   55.6s finished
Features: 2/5[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 172 tasks      | elapsed:    9.3s
[Parallel(n_jobs=2)]: Done 772 tasks      | elapsed:   33.5s
[Parallel(n_jobs=2)]: Done 1420 out of 1423 | elapsed:   59.0s remaining:    0.1s
[Parallel(n_jobs=2)]: Done 1423 out of 1423 | elapsed:   59.1s finished
Features: 3/5[Parallel(n

111

# Evaluate candidates

In [33]:
results = []
for i, candidate in enumerate(candidates):
    selector = candidate[3] 
    r = evaluate_candidate(fulldata, candidate[:3], n_tests=10)
    assert len(r) == 5, len(r)
    print('r', i, r[3], r[4])
    results.append((candidate, r, selector))

r 0 2.084718704223633 1.0248916149139404
r 1 6.966068267822266 24.16402244567871
r 2 15.762321710586548 29.390678882598877
r 3 0.19814515113830566 0.477435827255249
r 4 4.3473076820373535 2.2692883014678955
r 5 0.10947847366333008 0.4569723606109619
r 6 0.21562695503234863 0.5062267780303955
r 7 4.224099636077881 5.673119068145752
r 8 0.11412620544433594 0.4493098258972168
r 9 0.1999976634979248 0.4817383289337158
r 10 4.209441900253296 2.2678630352020264
r 11 0.11828470230102539 0.4352104663848877
r 12 0.19657182693481445 0.47269773483276367
r 13 4.245086908340454 2.2173268795013428
r 14 0.12880897521972656 0.4442102909088135
r 15 0.1919546127319336 0.4736363887786865
r 16 4.442005634307861 17.888440132141113
r 17 0.10945630073547363 0.41867780685424805
r 18 0.1938788890838623 0.47275352478027344
r 19 4.432848215103149 18.10808539390564
r 20 0.11419034004211426 0.4246981143951416
r 21 0.19411563873291016 0.47847437858581543
r 22 4.423448801040649 18.285876274108887
r 23 0.118894338607

In [34]:
test_scores = numpy.array([r[1][0] for r in results])
train_scores = numpy.array([r[1][1] for r in results])

def n_features(r):
    f = r[0][2]
    if f is None:
        return None
    else:
        return len(f)

def selected_params(r):
    cv_results = r[1][2]
    return pandas.DataFrame(cv_results).sort_values('mean_test_score').params[0]

df = pandas.DataFrame({
    'test_mean': test_scores.mean(axis=1),
    'test_std': test_scores.std(axis=1),
    'train_mean': train_scores.mean(axis=1),
    'train_std': train_scores.std(axis=1),
    'n_features': [ n_features(r) for r in results ],
    'features': [ r[0][2] for r in results ],
    'model': [ str(type(r[0][0]).__name__) for r in results ],
    'feature_selector': [ r[2] for r in results ],
    'gridsearch_time': [ r[1][3] for r in results ],
    'evaluation_time': [ r[1][4] for r in results ],
    'params': [ selected_params(r) for r in results ],
})
df.sort_values(by='test_mean', ascending=False).head(10)

,test_mean,test_std,train_mean,train_std,n_features,features,model,feature_selector,gridsearch_time,evaluation_time,params
24,0.921472,0.061590,0.942076,0.038910,5.0,"(28, 128, 138, 189, 279)",KNeighborsClassifier,sfs,0.198192,0.479904,{'n_neighbors': 1}
21,0.885327,0.063180,0.917890,0.050091,3.0,"(28, 128, 138)",KNeighborsClassifier,sfs,0.194116,0.478474,{'n_neighbors': 1}
1,0.878279,0.080978,0.919950,0.051033,NaN,None,RandomForestClassifier,all,6.966068,24.164022,"{'min_samples_leaf': 0.01, 'n_estimators': 10}"
25,0.873203,0.078614,0.898022,0.054612,5.0,"(28, 128, 138, 189, 279)",RandomForestClassifier,sfs,4.467008,18.177617,"{'min_samples_leaf': 0.01, 'n_estimators': 10}"
22,0.870601,0.074146,0.897546,0.040627,3.0,"(28, 128, 138)",RandomForestClassifier,sfs,4.423449,18.285876,"{'min_samples_leaf': 0.01, 'n_estimators': 10}"
13,0.849200,0.082641,0.902962,0.071322,5.0,"(103, 138, 444, 1354, 1391)",RandomForestClassifier,sfs,4.245087,2.217327,"{'min_samples_leaf': 0.01, 'n_estimators': 10}"
6,0.832461,0.070709,0.876923,0.059637,2.0,"(103, 138)",KNeighborsClassifier,sfs,0.215627,0.506227,{'n_neighbors': 1}
106,0.829501,0.071691,0.864949,0.057481,5.0,"[482, 565, 548, 837, 768]",RandomForestClassifier,random,4.293925,5.772554,"{'min_samples_leaf': 0.01, 'n_estimators': 10}"
12,0.814082,0.071508,0.877037,0.056838,5.0,"(103, 138, 444, 1354, 1391)",KNeighborsClassifier,sfs,0.196572,0.472698,{'n_neighbors': 1}
105,0.809731,0.080311,0.842623,0.074854,5.0,"[482, 565, 548, 837, 768]",KNeighborsClassifier,random,0.195692,0.473011,{'n_neighbors': 1}


In [ ]:
df.to_csv('summary.csv')

In [ ]:
## Best models

In [ ]:
df.hist('test_mean', bins=10, by=['n_features'], figsize=(10,10))

In [ ]:
df.hist('test_mean', bins=10, by=['model'], figsize=(10,10))

In [ ]:
df.hist('test_mean', bins=10, by=['feature_selector'], figsize=(10,10))